**Understanding ETL process**

In this example we are going to create a small ETL pipeline in python. 
To do that we are going to follow 3 steps :
1. Extract data from CSV file
2. Transform/Manipulate Data
3. Load Data into SQL Database

Download the CSV file using the requests library:

In [37]:
import requests

url = "https://raw.githubusercontent.com/diljeet1994/Python_Tutorials/master/Projects/Advanced%20ETL/crypto-markets.csv"
response = requests.get(url)

with open("data.csv", "w") as f:
    f.write(response.text)


Open the CSV file and do all the necessary transformations using the pandas library:

In [59]:
import pandas as pd

df = pd.read_csv("data.csv")
df.head()


,slug,asset,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,target-coin,TGT,Target Coin,29-09-2017,607,0.028961,0.054766,0.028961,0.041777,69996,0.0,0.4966,0.03
1,target-coin,TGT,Target Coin,30-09-2017,607,0.041783,0.046196,0.031435,0.031744,5725,0.0,0.0209,0.01
2,target-coin,TGT,Target Coin,01-10-2017,607,0.031761,0.035957,0.021040,0.028385,5012,0.0,0.4924,0.01
3,target-coin,TGT,Target Coin,02-10-2017,607,0.028375,0.054595,0.020417,0.022525,8010,0.0,0.0617,0.03
4,target-coin,TGT,Target Coin,03-10-2017,607,0.022527,0.032225,0.020211,0.020359,1787,0.0,0.0123,0.01


In this use case, we want to save only the data for Bitcoin BTC, Ripple XRP and calculates the average value of different price.

We only want the columns asset, name, date, open, close, high, low and a new column named ohlc_average.

In [60]:
assets = ["BTC", "XRP"]
df_filtered = df[df["asset"].isin(assets)]
df_filtered = df_filtered[['asset', 'name', 'date', 'open', 'high', 'low', 'close']]
df_filtered

,asset,name,date,open,high,low,close
878,BTC,Bitcoin,28-04-2013,135.300000,135.980000,132.100000,134.210000
879,BTC,Bitcoin,29-04-2013,134.440000,147.490000,134.000000,144.540000
880,BTC,Bitcoin,30-04-2013,144.000000,146.930000,134.050000,139.000000
881,BTC,Bitcoin,01-05-2013,139.000000,139.890000,107.720000,116.990000
882,BTC,Bitcoin,02-05-2013,116.380000,125.600000,92.280000,105.210000
...,...,...,...,...,...,...,...
1474,XRP,Ripple,15-01-2014,0.021182,0.022394,0.021122,0.022221
1475,XRP,Ripple,16-01-2014,0.022205,0.022343,0.021024,0.021024
1476,XRP,Ripple,17-01-2014,0.020990,0.021148,0.020098,0.020530
1477,XRP,Ripple,18-01-2014,0.020559,0.021374,0.020559,0.021278


The calculation of the average of the high, low, and close, is as follows:

OHLC Average = (Open + High + Low + Close) / 4



In [61]:
df_filtered["ohlc_average"] = (df_filtered["open"] + df_filtered["close"] + df_filtered["high"] + df_filtered["low"]) / 4
df_filtered.head()

,asset,name,date,open,high,low,close,ohlc_average
878,BTC,Bitcoin,28-04-2013,135.30,135.98,132.10,134.21,134.3975
879,BTC,Bitcoin,29-04-2013,134.44,147.49,134.00,144.54,140.1175
880,BTC,Bitcoin,30-04-2013,144.00,146.93,134.05,139.00,140.9950
881,BTC,Bitcoin,01-05-2013,139.00,139.89,107.72,116.99,125.9000
882,BTC,Bitcoin,02-05-2013,116.38,125.60,92.28,105.21,109.8675


Connect to the SQLite3 database and save the data to a table:

In [62]:
import sqlite3

 
conn = sqlite3.connect('crypto_database.db')
df_filtered.to_sql('crypto', conn, if_exists='replace', index=False)
conn.close()




In [68]:
conn = sqlite3.connect('crypto_database.db')
df = pd.read_sql_query('SELECT * FROM crypto', conn)
conn.close()
df

,asset,name,date,open,high,low,close,ohlc_average
0,BTC,Bitcoin,28-04-2013,135.300000,135.980000,132.100000,134.210000,134.397500
1,BTC,Bitcoin,29-04-2013,134.440000,147.490000,134.000000,144.540000,140.117500
2,BTC,Bitcoin,30-04-2013,144.000000,146.930000,134.050000,139.000000,140.995000
3,BTC,Bitcoin,01-05-2013,139.000000,139.890000,107.720000,116.990000,125.900000
4,BTC,Bitcoin,02-05-2013,116.380000,125.600000,92.280000,105.210000,109.867500
...,...,...,...,...,...,...,...,...
382,XRP,Ripple,15-01-2014,0.021182,0.022394,0.021122,0.022221,0.021730
383,XRP,Ripple,16-01-2014,0.022205,0.022343,0.021024,0.021024,0.021649
384,XRP,Ripple,17-01-2014,0.020990,0.021148,0.020098,0.020530,0.020692
385,XRP,Ripple,18-01-2014,0.020559,0.021374,0.020559,0.021278,0.020943


In [69]:
df[df["asset"] == "BTC"] 

,asset,name,date,open,high,low,close,ohlc_average
0,BTC,Bitcoin,28-04-2013,135.30,135.98,132.10,134.21,134.3975
1,BTC,Bitcoin,29-04-2013,134.44,147.49,134.00,144.54,140.1175
2,BTC,Bitcoin,30-04-2013,144.00,146.93,134.05,139.00,140.9950
3,BTC,Bitcoin,01-05-2013,139.00,139.89,107.72,116.99,125.9000
4,BTC,Bitcoin,02-05-2013,116.38,125.60,92.28,105.21,109.8675
...,...,...,...,...,...,...,...,...
213,BTC,Bitcoin,27-11-2013,923.85,1001.96,891.68,1001.96,954.8625
214,BTC,Bitcoin,28-11-2013,1003.38,1077.56,962.17,1031.95,1018.7650
215,BTC,Bitcoin,29-11-2013,1042.01,1146.97,1000.64,1131.97,1080.3975
216,BTC,Bitcoin,30-11-2013,1129.37,1156.14,1106.61,1129.43,1130.3875
